# Build Wells and Locations
I have to use data from the extracted meters to build the wells and locations tables before I can properly re-organized the extracted meters data into a table.

* Note: This imports extracted_meters.csv. I manually changed some rows in this csv on 7/11, if the csv is regenerated, those changes will be lost and may need to be re-done.

In [1]:
import pyodbc
import pandas as pd
import numpy as np

In [2]:
# Load extracted_meters
meters = pd.read_csv('./api/data/extracted_meters.csv')

The locations table will consist mostly of well locations, but I also need to include the monitoring wells and storage location(s).

Steps:
* Create initial locations and wells tables with monitoring wells and storage location(s)
* Create a list of wells from RA numbers in extracted data

In [3]:
monitoring_wells = {
    'PoeCorn':(33.613507745151,-104.519169218838),
    'TransWestern':(33.5924283752,-104.518961682916),
    'Berrendo-Smith':(33.438,-104.512),
    'LFD':(33.373029848637,-104.46614919662),
    'OrchardPark':(33.257,-104.415),
    'Greenfield':(33.16818643207846,-104.43742880081622),
    'Bartlett':(33.07014846801758,-104.37718963623047),
    'Cottonwood':(32.98,-104.521),
    'Zumwalt':(32.880372093723246,-104.42966573225806),
    'Artesia':(32.779409,-104.428663)
}
headquarters = (33.39467308099902, -104.49045710307567)


In [4]:
#Create initial locations dataframe with fields: name, type_id, latitude, longitude, TRSS, owner_id from monitoring_wells
locations_initial = pd.DataFrame.from_dict(monitoring_wells, orient='index', columns=['latitude','longitude'])

In [5]:
#Add on a row for headquarters
locations_initial.loc['headquarters'] = headquarters

In [6]:
locations_initial['type_id'] = 2
locations_initial.loc['headquarters','type_id'] = 1

Create a dataframe with the RA numbers from the extracted data. Only use values associated with installed meters to minimize duplicates.

In [7]:
ranumbers = meters[meters['meter_status_id'] == 1].loc[:,['sRANumber','sTRSS','latitude','longitude','owner_id']]

In [8]:
#Drop where sRANumber is null
ranumbers = ranumbers.dropna(subset=['sRANumber'])

I want to build the locations table from the ra number data but with unique locations as defined by TRSS. 

In [9]:
locations_ra = ranumbers.drop_duplicates(subset=['sTRSS'], keep='first')

In [10]:
# Change sRANumber to name and add on type_id = 2
locations_ra = locations_ra.rename(columns={'sRANumber':'name'})
locations_ra['type_id'] = 2

In [11]:
# Locations table is the concatenation of locations_initial and locations_ra
locations_initial['name'] = locations_initial.index
locations_table = pd.concat([locations_initial, locations_ra],ignore_index=True)

In [12]:
# Add on a location_id
locations_table['location_id'] = locations_table.index + 1

Now I will use all the RA numbers to create a wells table. Some will share locations, which might be wrong, but is the best I can do with the data I have.

In [13]:
#Drop sorted, duplicated RA numbers, keep first
ranumbers = ranumbers.sort_values('sRANumber').drop_duplicates('sRANumber', keep='first')

At this point there are still a few suspicious RA numbers. One is zero, the others have TRSS of an address. I will drop the zero but leave the others for now.

In [14]:
#Drop where RA number is 0
ranumbers.drop(2076, inplace=True)

In [18]:
# Add on a location_id to create the wells table
wells_table = ranumbers.merge(locations_table[['sTRSS','location_id']], how='left',left_on='sTRSS', right_on='sTRSS')

In [19]:
#Create a well_id from integer index on sorted table
wells_table = wells_table.sort_values('sRANumber')
wells_table['well_id'] = np.arange(1,len(wells_table)+1)

Save both tables to csv after reorganizing.

In [20]:
locations_table = locations_table[['location_id','name','type_id','sTRSS','latitude','longitude','owner_id']]
locations_table.to_csv('./api/data/locations.csv', index=False)

In [21]:
wells_table['name'] = wells_table['sRANumber']
wells_table = wells_table[['well_id','name','sRANumber','location_id']]
wells_table.to_csv('./api/data/wells.csv', index=False)

### Split up TRSS
I am adding this on after the fact...

In [2]:
# Load locations from csv
locations_table = pd.read_csv('./api/data/locations.csv')

In [3]:
test = '123.456.7.8'
test.split('.')

['123', '456', '7', '8']

In [4]:
# Create a new columns township, range, section, quarter from sTRSS split
locations_table['township'] = locations_table['sTRSS'].str.split('.').str[0]
locations_table['range'] = locations_table['sTRSS'].str.split('.').str[1]
locations_table['section'] = locations_table['sTRSS'].str.split('.').str[2]
locations_table['quarter'] = locations_table['sTRSS'].str.split('.').str[3]

In [7]:
# Replace non-numeric values in township, range, section, quarter with NaN
locations_table['township'] = pd.to_numeric(locations_table['township'], errors='coerce',downcast='integer')

In [8]:
locations_table['range'] = pd.to_numeric(locations_table['range'], errors='coerce',downcast='integer')
locations_table['section'] = pd.to_numeric(locations_table['section'], errors='coerce',downcast='integer')
locations_table['quarter'] = pd.to_numeric(locations_table['quarter'], errors='coerce',downcast='integer')

In [10]:
# Convert township, range, section, quarter to integer
locations_table['township'] = locations_table['township'].astype('Int64')
locations_table['range'] = locations_table['range'].astype('Int64')
locations_table['section'] = locations_table['section'].astype('Int64')
locations_table['quarter'] = locations_table['quarter'].astype('Int64')

In [13]:
# Ensure all land owner ids are integers
locations_table['owner_id'] = locations_table['owner_id'].astype('Int64')

In [14]:
# Export to csv
locations_table.to_csv('./api/data/locations.csv', index=False)